In [26]:
from os import listdir
from collections import Counter, defaultdict
from functools import reduce
import json
import math
from candidates_of_keyword import get_NC, getN_Crunk


In [40]:
def read_files(directory):
    pass

def get_field_counter(posts):
    limit = 450
    print("posts length: {}, only get: {} posts".format(len(posts), limit))
    
    clean_NC = []
    for post in posts[:limit]:
        nc = post['NC']
        nc = list(filter(lambda el: len(el.split(' ')) > 1, nc)) # filter unigram noun chunk
        clean_NC += nc
    return Counter(clean_NC)

def predict(scores):
    TEST_PATH = '/home/fun/Atos/final_test/data/testing_data/'
    files = listdir(TEST_PATH)
    for file in files:
        label = file
        
        doc = open(TEST_PATH + file, 'r', encoding='utf8').read()
        NC = get_NC(getN_Crunk(doc))
        pred_scores = defaultdict(lambda: 0)
        for nc in NC:
            for field in scores:
                if nc in scores[field]:
#                     print("field: {}, NC: {}, score: {}".format(field, nc, scores[field][nc]))
                    pred_scores[field] += scores[field][nc]
        
        predict = max(dict(pred_scores), key=lambda el: pred_scores[el])
        print("label: {},\tpredict: {}".format(label, predict))
#         print(pred_scores.items())

In [44]:
ls '/home/fun/Atos/final_test/data/can/'

can/  pl/  raw_data_post/  testing_data/


In [42]:
if __name__ == "__main__":
    KEYWORD_PATH = '/home/fun/Atos/final_test/li_data/can/'
    mode = 'total'
    
    ########### Get all fields counter and total counter
    fields_counter = {} 
    files = listdir(KEYWORD_PATH)
    for file in files:
        doc = json.loads(open(KEYWORD_PATH + file, 'r', encoding='utf8').read())
        for field in doc:
            cnt = get_field_counter(doc[field])
            
#             for k in list(cnt):
#                 if cnt[k] < 25: # threshold
#                     del cnt[k]
            
            fields_counter[field] = cnt # get_field_counter(doc[field])

    total_counter = reduce(lambda a, b: a + b, fields_counter.values())
    ############
    
    
    ############ Count tfidf
    scores = defaultdict(lambda: defaultdict(lambda: 0))
    word_num = len(total_counter.keys())
    total_count = sum(total_counter.values())
    for field in fields_counter:
        field_count = sum(fields_counter[field].values())
        other_count = total_count - field_count
        
        for key in fields_counter[field]:
            
            if mode == 'percentage':
                score = fields_counter[field][key] / total_counter[key]
            elif mode == 'total':
                score = fields_counter[field][key] / total_count
            elif mode == 'transform':
                p_w_f = math.log(fields_counter[field][key]+1 / field_count)
                p_w_o = math.log((total_counter[key] - fields_counter[field][key])+1 / other_count)
                score = p_w_f - p_w_o
            elif mode == 'tfidf': # hardcode
                tf = fields_counter[field][key] / field_count
                b=0
                if fields_counter["Front End Developer"][key] > 0: b+=1
                if fields_counter["Backend"][key] > 0: b+=1                
                idf = math.log(2/(b+1))
                score = tf*idf
            
            scores[field][key] = score
    ############ tfidf end
    
    predict(scores)

posts length: 394, only get: 450 posts
posts length: 776, only get: 450 posts
posts length: 831, only get: 450 posts
posts length: 433, only get: 450 posts
label: android.1.txt,	predict: android engineer
label: security.1.txt,	predict: security engineer
label: android.2.txt,	predict: android engineer
label: php.1.txt,	predict: security engineer
label: frontend.2.txt,	predict: Frontend
label: backend.1.txt,	predict: Frontend
label: security.2.txt,	predict: security engineer
label: frontend.1.txt,	predict: Frontend
label: backend.2.txt,	predict: security engineer


In [43]:
for field in scores:
    a = sorted(scores[field].items(), key=lambda k_v: k_v[1], reverse=True)[:3000]
    print("____________", field, "____________")
    print(a)

____________ Backend ____________
[('computer science', 0.0016999722453510963), ('web services', 0.0008673327782403553), ('spring boot', 0.0007285595337218984), ('sql server', 0.0007285595337218984), ('data structures', 0.0006244796003330558), ('core java', 0.0006244796003330558), ('software development', 0.0005550929780738274), ('software engineer', 0.0005203996669442131), ('apache camel', 0.0005203996669442131), ('related field', 0.00041631973355537054), ('continuous integration', 0.00041631973355537054), ('weekly payroll401k planaccount', 0.00041631973355537054), ('earn additional income', 0.00041631973355537054), ('dental insurancebi', 0.00041631973355537054), ('delivery manager', 0.00041631973355537054), ('experience working', 0.0003816264224257563), ('spring mvc', 0.0003816264224257563), ('client supportreferral program', 0.0003816264224257563), ('jee technologies', 0.0003816264224257563), ('spring framework', 0.0003469331112961421), ('api development', 0.0003122398001665279), ('

In [ ]:
word = 'node js'

print(fields_counter["Front End Developer"][word])
print(fields_counter["Backend"][word])
print(total_counter[word])

print(tfidf["Front End Developer"][word])
print(tfidf["Backend"][word])